In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1097605,2021-03-18,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1097606,2021-03-19,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1097607,2021-03-20,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1097608,2021-03-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
29599,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29601,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29603,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29605,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29607,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
678182,2021-03-18,Arkansas,Arkansas,5001,2020,35.00,5000,Arkansas,AR,Arkansas,State,3017804
678184,2021-03-19,Arkansas,Arkansas,5001,2023,35.00,5000,Arkansas,AR,Arkansas,State,3017804
678186,2021-03-20,Arkansas,Arkansas,5001,2024,35.00,5000,Arkansas,AR,Arkansas,State,3017804
678188,2021-03-21,Arkansas,Arkansas,5001,2024,35.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1097605,2021-03-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1097606,2021-03-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1097607,2021-03-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1097608,2021-03-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-22') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
426,2021-03-22,Snohomish,Washington,32145,553.00,53061,WA,County,822083,3910.19,67.27
850,2021-03-22,Cook,Illinois,488793,10144.00,17031,IL,County,5150233,9490.70,196.96
1273,2021-03-22,Orange,California,264849,4608.00,6059,CA,County,3175692,8339.88,145.10
1695,2021-03-22,Maricopa,Arizona,521585,9544.00,4013,AZ,County,4485414,11628.47,212.78
2117,2021-03-22,Los Angeles,California,1215330,22834.00,6037,CA,County,10039107,12105.96,227.45
...,...,...,...,...,...,...,...,...,...,...,...
1097089,2021-03-22,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1097250,2021-03-22,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1097380,2021-03-22,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1097506,2021-03-22,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
426,2021-03-22,Snohomish,Washington,32145,553.00,53061,WA,County,822083,3910.19,67.27,67.27,3910.19
850,2021-03-22,Cook,Illinois,488793,10144.00,17031,IL,County,5150233,9490.70,196.96,196.96,9490.70
1273,2021-03-22,Orange,California,264849,4608.00,6059,CA,County,3175692,8339.88,145.10,145.10,8339.88
1695,2021-03-22,Maricopa,Arizona,521585,9544.00,4013,AZ,County,4485414,11628.47,212.78,212.78,11628.47
2117,2021-03-22,Los Angeles,California,1215330,22834.00,6037,CA,County,10039107,12105.96,227.45,227.45,12105.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097089,2021-03-22,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1097250,2021-03-22,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1097380,2021-03-22,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1097506,2021-03-22,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
771631,2021-03-22,Clarke,Alabama,3476,57.00,1025,AL,County,23622,14715.10,241.30,241.30,14715.10,1
709061,2021-03-22,Hale,Alabama,2148,72.00,1065,AL,County,14651,14661.12,491.43,491.43,14661.12,2
555081,2021-03-22,Lowndes,Alabama,1356,52.00,1085,AL,County,9726,13942.01,534.65,534.65,13942.01,3
413226,2021-03-22,Franklin,Alabama,4180,82.00,1059,AL,County,31362,13328.23,261.46,261.46,13328.23,4
492288,2021-03-22,Etowah,Alabama,13608,339.00,1055,AL,County,102268,13306.22,331.48,331.48,13306.22,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676753,2021-03-22,Converse,Wyoming,981,17.00,56009,WY,County,13822,7097.38,122.99,122.99,7097.38,19
677113,2021-03-22,Sublette,Wyoming,688,7.00,56035,WY,County,9831,6998.27,71.20,71.20,6998.27,20
840858,2021-03-22,Lincoln,Wyoming,1309,12.00,56023,WY,County,19830,6601.11,60.51,60.51,6601.11,21
935444,2021-03-22,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
555081,2021-03-22,Lowndes,Alabama,1356,52.00,1085,AL,County,9726,13942.01,534.65,534.65,13942.01,3,1
709061,2021-03-22,Hale,Alabama,2148,72.00,1065,AL,County,14651,14661.12,491.43,491.43,14661.12,2,2
236419,2021-03-22,Walker,Alabama,7017,269.00,1127,AL,County,63521,11046.74,423.48,423.48,11046.74,27,3
554357,2021-03-22,Crenshaw,Alabama,1491,57.00,1041,AL,County,13772,10826.31,413.88,413.88,10826.31,30,4
490836,2021-03-22,Clay,Alabama,1480,54.00,1027,AL,County,13235,11182.47,408.01,408.01,11182.47,23,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597073,2021-03-22,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
840858,2021-03-22,Lincoln,Wyoming,1309,12.00,56023,WY,County,19830,6601.11,60.51,60.51,6601.11,21,20
819972,2021-03-22,Uinta,Wyoming,2109,12.00,56041,WY,County,20226,10427.17,59.33,59.33,10427.17,4,21
235312,2021-03-22,Teton,Wyoming,3530,9.00,56039,WY,County,23464,15044.32,38.36,38.36,15044.32,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.27,3910.19,15,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.27,3910.19,15,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.27,3910.19,15,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.27,3910.19,15,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.27,3910.19,15,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095022,2021-03-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1095023,2021-03-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1095024,2021-03-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1095025,2021-03-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
768693,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,241.30,14715.10,34,1,2.00
768694,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,241.30,14715.10,34,1,0.00
768695,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,241.30,14715.10,34,1,1.00
768696,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,241.30,14715.10,34,1,4.00
768697,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,241.30,14715.10,34,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854862,2021-03-18,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,3.00
854863,2021-03-19,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
854864,2021-03-20,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
854865,2021-03-21,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
552857,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13942.01,1,3,1.00,0.00
552858,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13942.01,1,3,0.00,0.00
552859,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13942.01,1,3,0.00,0.00
552860,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13942.01,1,3,0.00,0.00
552861,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13942.01,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551042,2021-03-18,Albany,Wyoming,3936,11.00,56001,WY,County,38880,10123.46,28.29,28.29,10151.75,23,7,2.00,0.00
551043,2021-03-19,Albany,Wyoming,3936,11.00,56001,WY,County,38880,10123.46,28.29,28.29,10151.75,23,7,0.00,0.00
551044,2021-03-20,Albany,Wyoming,3936,11.00,56001,WY,County,38880,10123.46,28.29,28.29,10151.75,23,7,0.00,0.00
551045,2021-03-21,Albany,Wyoming,3936,11.00,56001,WY,County,38880,10123.46,28.29,28.29,10151.75,23,7,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-22') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
282143,2021-03-22,Imperial,California,27267,698.00,6025,CA,County,181215,15046.77,385.18,385.18,15046.77,1,2,28.00,4.00
2117,2021-03-22,Los Angeles,California,1215330,22834.00,6037,CA,County,10039107,12105.96,227.45,227.45,12105.96,2,5,505.00,24.00
493692,2021-03-22,Inyo,California,1363,35.00,6027,CA,County,18039,7555.85,194.02,194.02,7555.85,3,24,1.00,0.00
140643,2021-03-22,San Bernardino,California,289846,4094.00,6071,CA,County,2180085,13295.17,187.79,187.79,13295.17,4,4,73.00,18.00
62965,2021-03-22,Stanislaus,California,58274,980.00,6099,CA,County,550660,10582.57,177.97,177.97,10582.57,5,9,134.00,1.00
78042,2021-03-22,Tulare,California,48832,807.00,6107,CA,County,466195,10474.59,173.10,173.10,10474.59,6,10,62.00,3.00
29217,2021-03-22,Riverside,California,293149,4141.00,6065,CA,County,2470546,11865.76,167.61,167.61,11865.76,7,6,182.00,24.00
50115,2021-03-22,San Joaquin,California,68855,1231.00,6077,CA,County,762148,9034.33,161.52,161.52,9034.33,8,19,141.00,2.00
28455,2021-03-22,Fresno,California,98211,1556.00,6019,CA,County,999101,9829.94,155.74,155.74,9829.94,9,14,99.00,24.00
598460,2021-03-22,Kings,California,22524,238.00,6031,CA,County,152940,14727.34,155.62,155.62,14727.34,10,3,13.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1041964,2021-03-22,Lassen,California,5637,1.00,6035,CA,County,30573,18437.84,3.27,3.27,18437.84,56,1,3.00,0.00
282143,2021-03-22,Imperial,California,27267,698.00,6025,CA,County,181215,15046.77,385.18,385.18,15046.77,1,2,28.00,4.00
598460,2021-03-22,Kings,California,22524,238.00,6031,CA,County,152940,14727.34,155.62,155.62,14727.34,10,3,13.00,0.00
140643,2021-03-22,San Bernardino,California,289846,4094.00,6071,CA,County,2180085,13295.17,187.79,187.79,13295.17,4,4,73.00,18.00
2117,2021-03-22,Los Angeles,California,1215330,22834.00,6037,CA,County,10039107,12105.96,227.45,227.45,12105.96,2,5,505.00,24.00
29217,2021-03-22,Riverside,California,293149,4141.00,6065,CA,County,2470546,11865.76,167.61,167.61,11865.76,7,6,182.00,24.00
176769,2021-03-22,Kern,California,105542,1170.00,6029,CA,County,900202,11724.26,129.97,129.97,11724.26,14,7,43.00,15.00
447481,2021-03-22,Merced,California,30257,429.00,6047,CA,County,277680,10896.36,154.49,154.49,10896.36,11,8,115.00,1.00
62965,2021-03-22,Stanislaus,California,58274,980.00,6099,CA,County,550660,10582.57,177.97,177.97,10582.57,5,9,134.00,1.00
78042,2021-03-22,Tulare,California,48832,807.00,6107,CA,County,466195,10474.59,173.10,173.10,10474.59,6,10,62.00,3.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
282143,2021-03-22,Imperial,California,27267,698.00,6025,CA,County,181215,15046.77,385.18,385.18,15046.77,1,2,28.00,4.00
2117,2021-03-22,Los Angeles,California,1215330,22834.00,6037,CA,County,10039107,12105.96,227.45,227.45,12105.96,2,5,505.00,24.00
493692,2021-03-22,Inyo,California,1363,35.00,6027,CA,County,18039,7555.85,194.02,194.02,7555.85,3,24,1.00,0.00
140643,2021-03-22,San Bernardino,California,289846,4094.00,6071,CA,County,2180085,13295.17,187.79,187.79,13295.17,4,4,73.00,18.00
62965,2021-03-22,Stanislaus,California,58274,980.00,6099,CA,County,550660,10582.57,177.97,177.97,10582.57,5,9,134.00,1.00
78042,2021-03-22,Tulare,California,48832,807.00,6107,CA,County,466195,10474.59,173.10,173.10,10474.59,6,10,62.00,3.00
29217,2021-03-22,Riverside,California,293149,4141.00,6065,CA,County,2470546,11865.76,167.61,167.61,11865.76,7,6,182.00,24.00
50115,2021-03-22,San Joaquin,California,68855,1231.00,6077,CA,County,762148,9034.33,161.52,161.52,9034.33,8,19,141.00,2.00
28455,2021-03-22,Fresno,California,98211,1556.00,6019,CA,County,999101,9829.94,155.74,155.74,9829.94,9,14,99.00,24.00
598460,2021-03-22,Kings,California,22524,238.00,6031,CA,County,152940,14727.34,155.62,155.62,14727.34,10,3,13.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,385.18,15046.77,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,385.18,15046.77,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,385.18,15046.77,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,385.18,15046.77,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,385.18,15046.77,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3692,03/18/21,Lassen,5634,1.00,18428.02,3.27,3.27,18437.84,56,1,2.00,0.00
3693,03/19/21,Lassen,5634,1.00,18428.02,3.27,3.27,18437.84,56,1,0.00,0.00
3694,03/20/21,Lassen,5634,1.00,18428.02,3.27,3.27,18437.84,56,1,0.00,0.00
3695,03/21/21,Lassen,5634,1.00,18428.02,3.27,3.27,18437.84,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,385.18,15046.77,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,385.18,15046.77,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,385.18,15046.77,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,385.18,15046.77,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,385.18,15046.77,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3775,03/18/21,Kings,22441,237.00,14673.07,154.96,155.62,14727.34,10,3,16.00,1.00
3776,03/19/21,Kings,22459,237.00,14684.84,154.96,155.62,14727.34,10,3,18.00,0.00
3777,03/20/21,Kings,22505,238.00,14714.92,155.62,155.62,14727.34,10,3,46.00,1.00
3778,03/21/21,Kings,22511,238.00,14718.84,155.62,155.62,14727.34,10,3,6.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)